<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Анализ" data-toc-modified-id="Анализ-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Анализ</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LinearRegression" data-toc-modified-id="LinearRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LinearRegression</a></span></li><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>DecisionTree</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RandomForest</a></span></li><li><span><a href="#LGBMRegressor" data-toc-modified-id="LGBMRegressor-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>LGBMRegressor</a></span></li><li><span><a href="#CastBoostRegressor" data-toc-modified-id="CastBoostRegressor-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>CastBoostRegressor</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

#  Прогнозирование заказов такси

Онлайн сервис заказа такси собрал исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. 

**Цель** - построить модель прогнозирования количества заказов такси на следующий час.

По техническому заданию значение метрики *RMSE* на тестовой выборке должно быть не больше 48. В качестве тестовой выборки использовать 10% от исходных данных.

## Подготовка данных

In [ ]:
# загружаем необходимые библиотеки
!pip install catboost
!pip install lightgbm

import pandas as pd
import matplotlib.pyplot as plt
import warnings

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV, TimeSeriesSplit, cross_val_score

from statsmodels.tsa.seasonal import seasonal_decompose

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

In [ ]:
RANDOM = 123

warnings.filterwarnings('ignore')

model_list = []
rmse_score_list = []
best_params = []

In [ ]:
#загружаем датасет
try:
    data = pd.read_csv('/datasets/taxi.csv')
except:
    data = pd.read_csv('/Users/amirk/Downloads/taxi.csv')

In [ ]:
data

In [ ]:
data.info()

In [ ]:
# столбец с датой переведем в формат datetime
data['datetime'] = pd.to_datetime(data['datetime'])

In [ ]:
# заменим индекс на столбец datetime
data.rename(index=data['datetime'], inplace=True, copy=False)

In [ ]:
# удалим столбец datetime
data.drop('datetime', axis=1, inplace=True)

In [ ]:
data

Данные загружены, пропусков нет! Ресемплируем данные по периоду 1 час. Нам важно знать количество заказов за час, а в данных заказы за 10 минут, поэтому применим функцию **sum()**.

In [ ]:
data = data.resample('1H').sum()

In [ ]:
data

In [ ]:
data.index.is_monotonic

Данные располложены в хронологическом порядке.    
Данные подготовлены к анализу!

## Анализ

In [ ]:
decomposed = seasonal_decompose(data)

In [ ]:
plt.plot(decomposed.seasonal['2018-03-01':'2018-03-05'])
plt.title('Ежедневные колебания частоты заказов')
plt.xlabel('Дата')
plt.ylabel('Условный уровень количества заказов')
plt.show

Видно четкую внутридневную зависимость изменения числа заказов. Рассмотрим более подробно внутридневные колебания. 

In [ ]:
plt.figure(figsize=[20,5])
plt.plot(decomposed.seasonal['2018-03-01'])
plt.title('Внутридневные колебания частоты заказов')
plt.xlabel('Дата')
plt.ylabel('Условный уровень количества заказов')
plt.show

Выявлены достаточно логичные зависимости. Пики заказов такси приходятся на полночь, когда прибывает большое количесвто рейсов, общественный транспорт прекращает свою работу. Затем количество заказов такси снижается, что вероятно связано с уменьшением количества прибывающий рейсов, и минимальное количество заказов приходится на 6 утра. Затем в течение суток отмечается постепенное нарастание количества заказов такси с формированием нескольких пиков в 10ч, 16-17ч и вновь полночь. Добавим данный параметр для прогнозирования

In [ ]:
plt.plot(decomposed.trend)
plt.title('Трендовая линия частоты заказов такси')
plt.xlabel('Дата')
plt.ylabel('Количество заказов')
plt.show

Общий тренд заказов такси указывает на более чем 3х кратное увеличение заказов такчи в период с марта по август 2018г, что может быть связано как с отпускным периодом, так и в целом с увеличением популярности сервиса заказа такси. Данный параметр будет полезной фичей для прогнозирования.

In [ ]:
plt.plot(decomposed.resid)

Видны отдельные выбросы, возможно связанные с каким-либо праздниками.

Напишем функцию для добавления фичей для прогнозирования.

In [ ]:
def make_features(data, shift_len, roll_len):    
    for i in range(1, shift_len):
        data['shift_{}'.format(i)] = data['num_orders'].shift(i)
    for i in range(2, roll_len):
        data['roll_{}'.format(i)] = data['num_orders'].shift(1).rolling(i).mean()
    data['day_of_week'] = data.index.dayofweek
    #data['month'] = data.index.month
    data['trend'] = decomposed.trend
    data['season'] = decomposed.seasonal
    data.dropna(inplace=True)
    return data

In [ ]:
data = make_features(data, 5, 3)

In [ ]:
data

## Обучение

Подготовим данные к загрузке в модель.

In [ ]:
X = data.drop('num_orders', axis=1)
y = data['num_orders']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

### LinearRegression

In [ ]:
def best_model_lr(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(cross_val_score(model,X_train, y_train, 
                           cv = TimeSeriesSplit(n_splits=8), 
                           n_jobs = -1, 
                           scoring = 'neg_root_mean_squared_error').mean()))
    best_params.append(model.get_params)
    print('Значение RMSE = ', abs(cross_val_score(model,X_train, y_train, 
                                               cv = TimeSeriesSplit(n_splits=8), 
                                               n_jobs = -1, 
                                               scoring = 'neg_root_mean_squared_error').mean()))
    return model 

In [ ]:
lr = best_model_lr(X_train, y_train)

In [ ]:
lr.predict(X_train)

In [ ]:
y_train

In [ ]:
def make_pict(model, y_train):    
    fig, ax1 = plt.subplots(figsize=(20,5))
    plt.plot(y_train['2018-03-01':'2018-03-03'], label='Реальные')
    plt.plot(pd.DataFrame(model.predict(X_train), index=y_train.index)['2018-03-01':'2018-03-03'], label='Предсказанные')
    plt.xlabel('Количество заказов', fontsize=16)
    plt.ylabel('Время', fontsize=16)
    plt.title('Количество заказов за 3 дня', fontsize=16)
    plt.legend(fontsize=16)
    plt.show()

In [ ]:
make_pict(lr, y_train)

### DecisionTree

In [ ]:
def best_model_dtr(X_train, y_train):
    model = DecisionTreeRegressor()
    params = {'max_depth':range(1,10), 'min_samples_split':range(2,10), 'min_samples_leaf':range(1,10)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=TimeSeriesSplit(n_splits=8), 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
dtr = best_model_dtr(X_train, y_train)

In [ ]:
make_pict(dtr, y_train)

### RandomForest

In [ ]:
# Для начала найдем лучшие параметры модели на небольшом количестве деревьев 
#с целью экономии времени обучения, а в целом можно и нужно количество деревьев сразу смотреть.
# Затем подберем такой гиперпараметр как количество деревьев
def best_model_rfc(X_train, y_train):
    model = RandomForestRegressor(random_state=RANDOM, n_estimators=10)
    params = {'max_depth':range(1,10), 'min_samples_split':range(2,10), 'min_samples_leaf':range(1,10)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=TimeSeriesSplit(n_splits=8), 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model = RandomForestRegressor(random_state=RANDOM)
    params = {'n_estimators':range(20,300,20), 
              'max_depth':[grid_model.best_params_['max_depth']],
              'min_samples_split':[grid_model.best_params_['min_samples_split']],
              'min_samples_leaf':[grid_model.best_params_['min_samples_leaf']]}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=TimeSeriesSplit(n_splits=8), 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
rfc = best_model_rfc(X_train, y_train)

In [ ]:
make_pict(rfc, y_train)

### LGBMRegressor

In [ ]:
def best_model_lgbm(X_train, y_train):
    model = LGBMRegressor()
    params = {'max_depth':range(1,15), 'min_data_in_leaf':range(1,40)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=TimeSeriesSplit(n_splits=8), 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=1, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
lgbm = best_model_lgbm(X_train, y_train)

In [ ]:
make_pict(lgbm, y_train)

### CastBoostRegressor

In [ ]:
def best_model_cbr(X_train, y_train):
    model = CatBoostRegressor(random_state=RANDOM)
    params = {'max_depth':range(2,10), 'iterations':range(20,1000, 20)}
    grid_model = RandomizedSearchCV(model, 
                                    params, 
                                    cv=5, 
                                    scoring='neg_root_mean_squared_error', 
                                    n_jobs=-1, 
                                    verbose=0, 
                                    random_state=RANDOM)
    grid_model.fit(X_train, y_train)
    model_list.append(model.__class__.__name__)
    rmse_score_list.append(abs(grid_model.best_score_))
    best_params.append(grid_model.best_params_)
    print('Значение RMSE = ', abs(grid_model.best_score_))
    print('Лучшие параметры модели:', grid_model.best_params_)
    return grid_model

In [ ]:
cbr = best_model_cbr(X_train, y_train)

In [ ]:
make_pict(cbr, y_train)

In [ ]:
# создадим сводную таблицу по результатам теста
top_list_df = pd.DataFrame({'Model':model_list, 
                            'RMSE':rmse_score_list,
                            'Best parameters':best_params }).sort_values(by='RMSE', ascending=True).reset_index(drop=True)
top_list_df

В ходе исследования нами было оценено 5 моделей для прогнозирования количества заказов такси на следующий час. Наилучший показатель RMSE оказался у модели LinearRegression. Именно данная модель будет применяться для прогнозирования заказов!

| Модель                | RMSE               | 
| :---:                 |    :----:          | 
| **LinearRegression**      | **22.22**              |
| LGBMRegressor | 22.40|
| CatBoostRegressor | 22.60 |
| RandomForestRegressor | 22.85 |
| DecisionTreeRegressor | 24.23              | 


Проверим работу модели на тестовых данных!

## Тестирование

In [ ]:
prediction = lr.predict(X_test)
final_rmse = mean_squared_error(y_test, prediction, squared=False)
final_rmse

Лучшей моделью для прогнозирования количества заказов такси признана модель **LinearRegression**. Итоговое значение RMSE на тестовых данных составило 40.37. Модель успешно прошла тестирование!

## Выводы

Онлайн сервису заказа такси была необходима  модель прогнозирования заказов такси на следующий час, для привлечения  большего количества водителей в период пиковой нагрузки. 

По техническому заданию значение метрики RMSE на тестовой выборке должно было быть не больше 48. В качестве тестовой выборки использовалось 10% от исходных данных.

В ходе исследования было выполнено:
- загрузка данных;
- переопределение типов данных в столбцах;
- ресемплирование временного ряда;
- оценка сезоных зависимостей и трендовой линии;
- добавление новых признаков;
- разделение данных на тренировочную и тестовую выборки;
- обучение моделей.

В ходе исследования нами было оценено 5 моделей для прогнозирования количества заказов такси на следующий час. Наилучший показатель RMSE оказался у модели LinearRegression. Именно данная модель будет применяться для прогнозирования заказов!

| Модель                | RMSE               | 
| :---:                 |    :----:          | 
| **LinearRegression**      | **22.22**              |
| LGBMRegressor | 22.40|
| CatBoostRegressor | 22.60 |
| RandomForestRegressor | 22.85 |
| DecisionTreeRegressor | 24.23              | 

Лучшей моделью для прогнозирования количества заказов такси признана модель **LinearRegression**. Итоговое значение RMSE на тестовых данных составило 40.37. 
